<a href="https://colab.research.google.com/github/PARAMKUSAMGOPISAISHANKAR/Handwriting-Text-Generation/blob/main/Handwriting_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed
from tensorflow.keras.models import Sequential
import string
import random
import os

def preprocess_text(text):
    """Preprocesses the text by converting to lowercase and removing unwanted characters."""
    text = text.lower()
    allowed_chars = string.ascii_lowercase + string.digits + " .,!?\n"
    text = ''.join(c for c in text if c in allowed_chars)
    return text

def create_char_mappings(text):
    """Creates character-to-index and index-to-character mappings."""
    chars = sorted(list(set(text)))
    char_to_index = {c: i for i, c in enumerate(chars)}
    index_to_char = {i: c for i, c in enumerate(chars)}
    return chars, char_to_index, index_to_char

def create_sequences(text, seq_length, char_to_index):
    """Creates input-output sequences for training."""
    input_chars = []
    output_chars = []
    for i in range(0, len(text) - seq_length):
        input_seq = text[i:i + seq_length]
        output_seq = text[i + seq_length]
        input_chars.append([char_to_index[char] for char in input_seq])
        output_chars.append(char_to_index[output_seq])
    return np.array(input_chars), np.array(output_chars)

def create_model(vocab_size, seq_length):
    """Creates the RNN model."""
    model = Sequential([
        LSTM(256, input_shape=(seq_length, 1), return_sequences=True),
        LSTM(256),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
    return model

def generate_text(model, seed_text, seq_length, num_chars_to_generate, char_to_index, index_to_char):
    """Generates text from the trained model."""
    generated_text = seed_text
    for _ in range(num_chars_to_generate):
        input_eval = [char_to_index[char] for char in seed_text]
        input_eval = np.expand_dims(input_eval, axis=0)
        input_eval = np.expand_dims(input_eval, axis=-1) # add dimension for LSTM input

        predictions = model.predict(input_eval, verbose=0)
        predicted_id = np.random.choice(len(predictions[0]), p=predictions[0]) #Sample from the distribution.
        predicted_char = index_to_char[predicted_id]

        generated_text += predicted_char
        seed_text = seed_text[1:] + predicted_char

    return generated_text

def train_and_generate(file_path, seq_length=100, epochs=20, generate_length=500):
    """Trains the model and generates text."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return

    text = preprocess_text(text)
    chars, char_to_index, index_to_char = create_char_mappings(text)
    vocab_size = len(chars)
    input_chars, output_chars = create_sequences(text, seq_length, char_to_index)

    input_chars = np.expand_dims(input_chars, axis=-1) # add dimension for LSTM input

    model = create_model(vocab_size, seq_length)
    model.fit(input_chars, output_chars, epochs=epochs, batch_size=128)

    start_index = random.randint(0, len(text) - seq_length - 1)
    seed_text = text[start_index:start_index + seq_length]
    generated_text = generate_text(model, seed_text, seq_length, generate_length, char_to_index, index_to_char)

    print("\nGenerated Text:\n", generated_text)

# Example Usage
# Create a text file named 'handwriting.txt' with sample handwritten-like text.
# For example, you can use a text file containing paragraphs of text.
# Then run:

if __name__ == "__main__":
    # Create a dummy text file if one does not exist.
    if not os.path.exists("handwriting.txt"):
      with open("handwriting.txt", "w") as f:
        f.write("The quick brown fox jumps over the lazy dog. This is a sample text for handwriting generation. Hello world.")

    train_and_generate("handwriting.txt")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 3.3077
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 2.5796
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 668ms/step - loss: 2.2169
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 778ms/step - loss: 2.0499
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 623ms/step - loss: 1.9898
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 695ms/step - loss: 1.9442
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 660ms/step - loss: 1.9140
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 724ms/step - loss: 1.8901
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 1.8585
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 636ms/step - loss: 1.8281
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 1.7934
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 939ms/step - loss: 1.7430
Epoch 13/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 997ms/step - loss: 1.6893
Epoch 14/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 871ms/step - loss: 1.6317
Epoch 15/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 915ms/step - loss: 1.6469
Epoch 16/20
1/1 ━━━━━━━━━━━━━━━